# Momentum Shift Score: Quantifying Career-Altering Moments in Major League Baseball

### Objectives:
This analysis focuses on identifying and evaluating pivotal moments in baseball games—specifically, events involving a pitched ball that result in the greatest changes in Win Probability Added (WPA). The goal is to assess how these moments influence the career trajectories of both the pitcher and the hitter. By quantifying this phenomenon into a single metric, we aim to provide a comprehensive narrative of how critical moments can shape the careers of the players involved.

### Import Libraries
Used pybaseball to scrape MLB pitch data throughout many pivotal moments in players' careers

In [17]:
from pybaseball import statcast
import pybaseball
import numpy as np

# Analysis 1
## Pete Alonso HR off of Devin Williams
### 2024 NL Wild Card - October 3

Pull data from games played on October 3, 2024 by the Milwalkee Brewers

In [11]:
mets_brewers_wc = statcast('2024-10-03', team='MIL')
mets_brewers_wc.head()

This is a large query, it may take a moment to complete


  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/envs/baseball/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 1/1 [00:00<00:00,  9.73it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
131,SL,2024-10-03,87.3,-2.71,5.53,"Ross, Joe",621438,605452,field_out,hit_into_play,...,1,3,1,1,<NA>,2,2.69,-0.55,-0.55,33.1
33,FF,2024-10-03,94.3,-2.23,5.26,"Williams, Devin",516782,642207,single,hit_into_play,...,1,3,1,1,<NA>,2,1.2,0.86,0.86,28.5
34,CH,2024-10-03,86.3,-2.58,5.01,"Williams, Devin",516782,642207,NaN,swinging_strike,...,1,3,1,1,<NA>,2,3.2,1.67,1.67,18.9
36,CH,2024-10-03,86.4,-2.61,5.05,"Williams, Devin",516782,642207,NaN,swinging_strike,...,1,3,1,1,<NA>,2,3.22,1.74,1.74,15.9
39,CH,2024-10-03,87.5,-2.49,5.06,"Williams, Devin",516782,642207,NaN,ball,...,1,3,1,1,<NA>,2,2.82,1.57,1.57,19.4


### Find Pete Alonso's ID

A simple eye search is all that was needed to find the infamous homer on this day.
After finding the only HR hit off of Devin Williams we conclude Alonso's ID is as follows.

In [12]:
pete_alonso_id = 624413

### Filter Alonso's at bat vs. Devin Williams

Filter the game data by pitcher name and batter ID to narrow the data frame to the infamous home run at bat

In [18]:
alonso_williams_ab = mets_brewers_wc[
    (mets_brewers_wc['player_name'] == 'Williams, Devin') & 
    (mets_brewers_wc['batter'] == pete_alonso_id)
]
alonso_williams_ab

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
61,CH,2024-10-03,86.1,-2.56,5.15,"Williams, Devin",624413,642207,home_run,hit_into_play,...,1,3,1,1,<NA>,2,3.25,1.68,1.68,19.9
63,CH,2024-10-03,86.5,-2.61,5.05,"Williams, Devin",624413,642207,NaN,ball,...,1,3,1,1,<NA>,2,3.13,1.74,1.74,17.8
64,FF,2024-10-03,94.0,-2.3,5.35,"Williams, Devin",624413,642207,NaN,ball,...,1,3,1,1,<NA>,2,1.03,0.74,0.74,31.2
67,FF,2024-10-03,93.3,-2.33,5.29,"Williams, Devin",624413,642207,NaN,ball,...,1,3,1,1,<NA>,2,1.26,0.95,0.95,29.2
68,CH,2024-10-03,87.1,-2.59,5.15,"Williams, Devin",624413,642207,NaN,called_strike,...,1,3,1,1,<NA>,2,3.13,1.7,1.7,18.9


# Finding pivotal moments

In [19]:
pybaseball.cache.enable()

data = statcast('2024-03-20', '2024-11-10')
data.head

This is a large query, it may take a moment to complete


100%|██████████| 236/236 [00:06<00:00, 38.35it/s]
/opt/anaconda3/envs/baseball/lib/python3.12/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


<bound method NDFrame.head of      pitch_type  game_date  release_speed  release_pos_x  release_pos_z  \
161          KC 2024-10-30           77.5          -1.11           5.65   
170          KC 2024-10-30           78.7          -1.01           5.73   
179          FC 2024-10-30           93.1          -1.19           5.53   
187          KC 2024-10-30           78.5          -1.19            5.7   
205          KC 2024-10-30           77.4          -1.23           5.78   
...         ...        ...            ...            ...            ...   
2046         FF 2024-03-20           95.7          -2.13           5.77   
2206         SI 2024-03-20           93.9          -2.17            5.7   
2329         SI 2024-03-20           93.4           -2.0            5.7   
2516         FF 2024-03-20           92.6          -1.94           5.83   
2700         FF 2024-03-20           94.5          -1.79           5.83   

          player_name  batter  pitcher     events              descri

In [25]:
wpa = np.array(data['delta_home_win_exp'])
wpa.max(), wpa.min()

highest_wpa = data.loc[data['delta_home_win_exp'].abs() >= 0.5]
highest_wpa = highest_wpa[['game_date', 'player_name', 'batter', 'pitcher', 'events', 'delta_home_win_exp']]
highest_wpa.head()

,game_date,player_name,batter,pitcher,events,delta_home_win_exp
291,2024-10-25,"Cortes, Nestor",518692,641482,home_run,0.729
154,2024-10-03,"Williams, Devin",624413,642207,home_run,-0.661
451,2024-09-30,"Johnson, Pierce",596019,572955,home_run,-0.657
350,2024-09-30,"Díaz, Edwin",645277,621242,double,0.643
3941,2024-09-28,"Miller, Ryan",680716,668943,single,-0.584
